# Analysis of Wine Quality and Prediction Using Logistic Regression

by Alix, Paramveer, Susannah, Zoe 2024/11/23

In [1]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo 
from sklearn.model_selection import train_test_split
import altair as alt
import altair_ally as aly

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

## Summary

(write something here)

## Introduction

The quality of wine is influenced by various chemical properties and sensory factors that determine its taste, aroma, and overall acceptability. Here, we aim to predict the quality of wine using a publicly available wine quality dataset. Machine learning-based predictive modeling is commonly used in the field of wine quality to identify patterns and relationships in key features such as alcohol, sulfates, and volatile acidity, which are critical factors impacting wine quality(Jain et al. 2023). By applying machine learning model, we seek to enhance the accuracy of wine quality predictions and contribute to the advancement of data-driven approaches in wine evaluation methodologies.

## Methods

### Data

The dataset used in this project is the Wine Quality dataset from the UCI Machine Learning Repository (Cortez et al. 2009) and can be found here: https://archive.ics.uci.edu/dataset/186/wine+quality. These datasets are related to red and white variants of the Portuguese "Vinho Verde" wine. They contains physicochemical properties (e.g., acidity, sugar content, and alcohol) of different wine samples, alongside a sensory score representing the quality of the wine, rated by experts on a scale from 0 to 10. Each row in the dataset represents a wine sample, with the columns detailing 11 physicochemical attributes and the quality score. The classes are ordered and not balanced (e.g. there are many more normal wines than excellent or poor ones).

Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.).

#### 0. Import the dataset and inspect the data

In [2]:
# Get the complete dataset
wine_quality = fetch_ucirepo(id=186)
raw_data = wine_quality.data.original 

# Save data into data folder as well
raw_data.to_csv('../data/wine_quality.csv', index=False)

# reorder columns
raw_data['quality'] = raw_data.pop('quality')
  
# Split training and testing data
train_df, test_df = train_test_split(raw_data, test_size=0.2, random_state=522)

# Store split data in data folder


train_df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,color,quality
5323,5.4,0.255,0.33,1.2,0.051,29.0,122.0,0.99048,3.37,0.66,11.3,white,6
364,12.8,0.615,0.66,5.8,0.083,7.0,42.0,1.00220,3.07,0.73,10.0,red,7
4158,5.7,0.255,0.65,1.2,0.079,17.0,137.0,0.99307,3.20,0.42,9.4,white,5
5634,6.4,0.200,0.22,7.4,0.032,53.0,172.0,0.99404,3.24,0.58,11.0,white,6
2691,7.1,0.210,0.40,1.2,0.069,24.0,156.0,0.99280,3.42,0.43,10.6,white,6


In [3]:
# Check data info
print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")
print('-'*50)
train_df.info()

Training data shape: (5197, 13)
Testing data shape: (1300, 13)
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 5197 entries, 5323 to 3988
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         5197 non-null   float64
 1   volatile_acidity      5197 non-null   float64
 2   citric_acid           5197 non-null   float64
 3   residual_sugar        5197 non-null   float64
 4   chlorides             5197 non-null   float64
 5   free_sulfur_dioxide   5197 non-null   float64
 6   total_sulfur_dioxide  5197 non-null   float64
 7   density               5197 non-null   float64
 8   pH                    5197 non-null   float64
 9   sulphates             5197 non-null   float64
 10  alcohol               5197 non-null   float64
 11  color                 5197 non-null   object 
 12  quality               5197 non-null   int64  
dtypes: float64(

In [4]:
# Data description
train_df.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
count,5197.000000,5197.000000,5197.000000,5197.000000,5197.000000,5197.000000,5197.000000,5197.000000,5197.000000,5197.000000,5197.000000,5197.000000
mean,7.197970,0.338300,0.317474,5.453252,0.055704,30.509140,116.052145,0.994672,3.218888,0.529623,10.486252,5.825284
std,1.274209,0.164158,0.144139,4.702806,0.034909,17.393583,56.316933,0.002928,0.161254,0.147649,1.188889,0.872927
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,78.000000,0.992300,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,119.000000,0.994800,3.210000,0.510000,10.300000,6.000000
75%,7.600000,0.400000,0.390000,8.100000,0.064000,41.000000,156.000000,0.996950,3.320000,0.600000,11.300000,6.000000
max,15.600000,1.580000,1.230000,26.050000,0.611000,146.500000,366.500000,1.003690,4.010000,2.000000,14.000000,9.000000


**Preprocessing data requirements**

From the data info and description, we can see that:
1. The numerical features are in different scales, we need to normalize them.
2. There is one categorical feature: 'color', we need to encode it.

#### 1.EDA

**1.1 Distribution of quality scores across numerical features**

In [5]:
aly.alt.data_transformers.enable('vegafusion')

aly.dist(train_df, color='quality')

RuntimeError: The versions of the vegafusion and vegafusion-python-embed packages must match
and must be version 1.5.0 or greater.
Found:
 - vegafusion==2.0.0rc1
 - vegafusion-python-embed==1.6.9


alt.ConcatChart(...)

From the distribution plots above, we have the following findings:
1. Higher quality wines tend to have higher alcohol content
2. Higher quality wines generally have lower volatile acidity
3. pH seems to have little discrimination power for quality (all quality levels overlap significantly)
4. The `density` feature does not showing any meaningful relationship with wine quality

**1.2 Distribution of quality scores by categorical feature (wine color)**

In [6]:
# Calculate the proportions of each quality score for different wine colors
proportions = (train_df.groupby(['color', 'quality'])
              .size()
              .reset_index(name='count')
              .assign(proportion=lambda x: x.groupby('color')['count'].transform(lambda y: y / y.sum()))
              .reset_index(drop=True))

# Create a line plot showing the proportions of each quality score for different wine colors
alt.Chart(proportions).mark_line(
    interpolate='monotone',  
    point=True,             
    tension=0.7,           
    strokeWidth=2          
).encode(
    x=alt.X('quality:Q', 
            title='Quality Score',
            scale=alt.Scale(domain=[2.5, 9.5])),
    y=alt.Y('proportion:Q', 
            title='Proportion', 
            axis=alt.Axis(format='.0%')),
    color=alt.Color('color:N', 
                   title='Wine Type',
                   scale=alt.Scale(domain=['red', 'white'],
                                 range=['#1f77b4', '#ff7f0e'])), 
    tooltip=[
        alt.Tooltip('quality:Q', title='Quality'),
        alt.Tooltip('proportion:Q', title='Proportion', format='.1%'),
        alt.Tooltip('color:N', title='Wine Type')
    ]
).properties(
    width=500,
    height=300
)

RuntimeError: The versions of the vegafusion and vegafusion-python-embed packages must match
and must be version 1.5.0 or greater.
Found:
 - vegafusion==2.0.0rc1
 - vegafusion-python-embed==1.6.9


alt.Chart(...)

This plot simply shows that white wine in average tends to have higher quality scores than red wine.

**1.3 Correlation matrix**

In [7]:
aly.corr(train_df)

RuntimeError: The versions of the vegafusion and vegafusion-python-embed packages must match
and must be version 1.5.0 or greater.
Found:
 - vegafusion==2.0.0rc1
 - vegafusion-python-embed==1.6.9


alt.ConcatChart(...)

As shown above, it seems that the correlation between total sulfur dioxide and free sulfur dioxide is high, we might want to use one of them to represent the other. But let's see the scatter plot for these two features first.

In [8]:
# Create scatter plot with regression line
alt.Chart(train_df[['free_sulfur_dioxide', 'total_sulfur_dioxide']].sample(600)).mark_circle().encode(
    x='free_sulfur_dioxide',
    y='total_sulfur_dioxide'
).properties(
    width=300,
    height=200
) + alt.Chart(
    train_df[['free_sulfur_dioxide', 'total_sulfur_dioxide']].sample(600)
).mark_line(color='red').encode(
    x='free_sulfur_dioxide',
    y='total_sulfur_dioxide'
).transform_regression(
    'free_sulfur_dioxide', 
    'total_sulfur_dioxide'
)

RuntimeError: The versions of the vegafusion and vegafusion-python-embed packages must match
and must be version 1.5.0 or greater.
Found:
 - vegafusion==2.0.0rc1
 - vegafusion-python-embed==1.6.9


alt.LayerChart(...)

From the scatter plot, we can see that there is a positive linear correlation between between free and total sulfur dioxide, but the relationship is not perfectly linear. Since keeping both features would not make the model too complex, we will leave them both in the model for now.



**1.4 Outlier detection**

In [9]:
# Get numerical columns only (exclude 'quality' and 'color')
numerical_cols = train_df.select_dtypes(include=['float64', 'int64']).columns
numerical_cols = [col for col in numerical_cols if col != 'quality']

# Create box plots
charts = []
for col in numerical_cols:
    chart = alt.Chart(train_df).mark_boxplot().encode(
        x=alt.X(col + ':Q', scale=alt.Scale(zero=False)),
        y=alt.Y('color:N', title=None),  # keep color but add title
        color=alt.Color('color:N', legend=alt.Legend(title="Wine Type"))
    ).properties(
        title=col,
        width=250,
        height=80
    )
    charts.append(chart)

# Display all the box plots together
n_cols = 3
n_rows = (len(charts) + n_cols - 1) // n_cols
grid = alt.vconcat(*[alt.hconcat(*charts[i:i+n_cols]) for i in range(0, len(charts), n_cols)])

grid

RuntimeError: The versions of the vegafusion and vegafusion-python-embed packages must match
and must be version 1.5.0 or greater.
Found:
 - vegafusion==2.0.0rc1
 - vegafusion-python-embed==1.6.9


alt.VConcatChart(...)

From the box plots above, we have the following findings:

1. Outliers:
   - Many features show significant outliers
   - Particularly noticeable in sulfur dioxide and residual sugar

1. Distributions:
   - Most features show right-skewed distributions
   - pH shows relatively normal distribution for both types

**1.5 The distribution of the target variable(quality)**

In [10]:
# Create a DataFrame with the quality counts
quality_df = pd.DataFrame({
    'quality': train_df['quality'].value_counts().index,
    'count': train_df['quality'].value_counts().values
})
quality_df['percentage'] = (quality_df['count'] / len(train_df) * 100).round(1)
quality_df = quality_df.sort_values('quality')

# Create the bar plot
chart = alt.Chart(quality_df).mark_bar().encode(
    x=alt.X('quality:O', title='Quality Score'),
    y=alt.Y('percentage:Q', title='Percentage (%)')
).properties(
    width=350,
    height=200,
    title='Distribution of Wine Quality Scores'
)

chart

RuntimeError: The versions of the vegafusion and vegafusion-python-embed packages must match
and must be version 1.5.0 or greater.
Found:
 - vegafusion==2.0.0rc1
 - vegafusion-python-embed==1.6.9


alt.Chart(...)

We can see our target variable has a normal distribution. The scores are centered around 5-6, with symmetric decreasing frequencies on both sides, forming a classic bell-shaped curve.

### Analysis

The Logistic Regression algorithm was used to build a classification model to predict the quality as an ordinal and numeric integer (found in the `quality` column of the data set). All variables included in the original data set, including wine color (i.e. red or white) were used to fit the model. Data was split with 80% being partitioned into the training set and 20% being partitioned into the test set. The hyperparameter C was chosen using 5-fold cross validation with the accuracy score as the classification metric. All variables were standardized just prior to model fitting. `color` column is converted to a binary column named `is_red` which has value of 1 for red wine, and 0 for white wine.

## Results and Discussion

We split and transform the data (i.e. wine color into binary variable and using standard scalers for all other features) and build our logistic regression model:

In [ ]:
# Split dataset
X_train, X_test, y_train, y_test = (train_df.drop(columns='quality'), test_df.drop(columns='quality'),
                                    train_df['quality'], test_df['quality']
                                    )

numeric_features = X_train.select_dtypes(include='number').columns.tolist()
binary_features = ['color']

# Make column transformer
preprocessor = make_column_transformer(
    (OneHotEncoder(drop='if_binary'), binary_features),
    (StandardScaler(), numeric_features)
)

# Make pipeline using StandardScaler and LogisticRegression
model = make_pipeline(
    preprocessor,
    LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
)

We find the best hyperparamter C for the model:

In [12]:
# Define parameter distribution
param_dist = {
    'logisticregression__C': stats.uniform(0.001, 100),
}

# Perform randomized search
random_search = RandomizedSearchCV(model, param_distributions=param_dist,
                                   cv=5, n_iter=50,
                                   scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", random_search.best_params_)

c:\Users\param\miniforge3\envs\522\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Best Parameters: {'logisticregression__C': np.float64(86.61861457749352)}


With our tuned model using the best C hyperparameter found above, we find the accuracy score of our predictions, comparing them to actual wine quality in the test set:

In [13]:
# Evaluate
y_pred = random_search.predict(X_test)
# print(classification_report(y_test, y_pred))
accuracy_score(y_test, y_pred)

0.5230769230769231

While the performance of this model is not likely very useful in predicting wine quality, as we observed an accuracy score of 0.52, we gained insights on directions that could be further explored. First, we chose logistic regression as it is an intuitive first-step to approach a dataset with largely numeric features representing measurements of contents inside wines. Therefore, further analysis inspecting presence of linear relationships can be conducted using logistic regression results. We can then propose another model, e.g.Tree-based ones like Random Forest, to see whether it does better in wine quality prediction should there be weak linear relationships observed. Second, data cleaning might benefit our decision in choosing an optimal model as outliers have been widely observed across many features, according to our EDA in the previous section. It might be worth it to understand what all features represent and apply human knowledge to modify and "treat" the data so that it is more suitable for training than how it is currently presented. This involves speaking with professionals that understand wine makeup and qualities and seek their insights on reasons of outlier presence and their indications. We believe conducting the above two next-steps will give us a better knowledge foundation in order for us to choose a model that performs better in the future.

## References

Jain, K., Kaushik, K., Gupta, S.K. et al. Machine learning-based predictive modelling for the enhancement of wine quality. Sci Rep 13, 17042 (2023). https://doi.org/10.1038/s41598-023-44111-9

Cortez, P., Cerdeira, A., Almeida, F., Matos, T., & Reis, J. (2009). Wine Quality [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C56S3T.